# Additional Cleaning

In [3]:
import os
import pandas as pd

In [4]:
# Folder containing Parquet files
folder_path = 'C:\\Users\\clair\\MasterThesis\\Processed_weather_data\\'

# List all Parquet files in the folder
parquet_files = [file for file in os.listdir(folder_path) if file.endswith('.parquet')]

# Dictionary to store DataFrames
dfs = {}

# Read each Parquet file into a DataFrame
for parquet_file in parquet_files:
    location_name = parquet_file.split('_')[-1].split('.')[0]  # Extract location name from file name
    file_path = os.path.join(folder_path, parquet_file)
    dfs[location_name] = pd.read_parquet(file_path)

In [5]:
# Extract DataFrames from the dictionary
df_canyons = dfs['Canyons Ski Resort']
df_las_vegas = dfs['Las Vegas']
df_miami_beach = dfs['Miami Beach']
df_disney = dfs['Walt Disney Parks & Resorts']

In [6]:
df_canyons.describe

<bound method NDFrame.describe of        Canyons Ski Resort_STATION_NUM Canyons Ski Resort_STATION_ID  \
0                                   4                           SNC   
1                                   4                           SNC   
2                                   4                           SNC   
3                                   4                           SNC   
4                                   4                           SNC   
...                               ...                           ...   
102250                         170915                         CTBST   
102251                         170915                         CTBST   
102252                         170915                         CTBST   
102253                         170915                         CTBST   
102254                         170915                         CTBST   

       Canyons Ski Resort_DATETIME  Canyons Ski Resort_air_temp_set_1  \
0              2019-06-01 08:00:00      

In [7]:
df_las_vegas.describe

<bound method NDFrame.describe of        Las Vegas_STATION_NUM Las Vegas_STATION_ID  Las Vegas_DATETIME  \
0                        436                 KVGT 2019-06-01 08:00:00   
1                        436                 KVGT 2019-06-01 09:00:00   
2                        436                 KVGT 2019-06-01 10:00:00   
3                        436                 KVGT 2019-06-01 11:00:00   
4                        436                 KVGT 2019-06-01 12:00:00   
...                      ...                  ...                 ...   
298677                199983                OLVAB 2023-06-30 14:00:00   
298678                199983                OLVAB 2023-06-30 15:00:00   
298679                199983                OLVAB 2023-06-30 16:00:00   
298680                199983                OLVAB 2023-06-30 17:00:00   
298681                199983                OLVAB 2023-06-30 18:00:00   

        Las Vegas_air_temp_set_1  Las Vegas_pressure_set_1  \
0                      24.6

In [8]:
df_miami_beach.describe

<bound method NDFrame.describe of       Miami Beach_STATION_NUM Miami Beach_STATION_ID Miami Beach_DATETIME  \
0                       34573                  E0888  2019-06-01 08:00:00   
1                       34573                  E0888  2019-06-01 09:00:00   
2                       34573                  E0888  2019-06-01 10:00:00   
3                       34573                  E0888  2019-06-01 11:00:00   
4                       34573                  E0888  2019-06-01 12:00:00   
...                       ...                    ...                  ...   
27485                   34573                  E0888  2023-06-30 13:00:00   
27486                   34573                  E0888  2023-06-30 14:00:00   
27487                   34573                  E0888  2023-06-30 15:00:00   
27488                   34573                  E0888  2023-06-30 16:00:00   
27489                   34573                  E0888  2023-06-30 17:00:00   

       Miami Beach_air_temp_set_1 Miami B

In [9]:
df_disney.describe

<bound method NDFrame.describe of        Walt Disney Parks & Resorts_STATION_NUM  \
0                                         4851   
1                                         4851   
2                                         4851   
3                                         4851   
4                                         4851   
...                                        ...   
247369                                  177048   
247370                                  177048   
247371                                  177048   
247372                                  177048   
247373                                  177048   

       Walt Disney Parks & Resorts_STATION_ID  \
0                                        KISM   
1                                        KISM   
2                                        KISM   
3                                        KISM   
4                                        KISM   
...                                       ...   
247369                

In [11]:
# Display the structure of the data frames
data = {
    'DataFrame': ['df_canyons', 'df_miami_beach', 'df_las_vegas', 'df_disney'],
    'Rows': [df_canyons.shape[0], df_miami_beach.shape[0], df_las_vegas.shape[0], df_disney.shape[0]],
    'Columns': [df_canyons.shape[1], df_miami_beach.shape[1], df_las_vegas.shape[1], df_disney.shape[1]]
}

df_summary = pd.DataFrame(data)

print(df_summary)

        DataFrame    Rows  Columns
0      df_canyons  102255       54
1  df_miami_beach   27490       54
2    df_las_vegas  298682       54
3       df_disney  247374       54


In [12]:
# Drop columns with all NaN values
df_canyons_cleaned = df_canyons.dropna(axis=1, how='all')
df_miami_cleaned = df_miami_beach.dropna(axis=1, how='all')
df_vegas_cleaned = df_las_vegas.dropna(axis=1, how='all')
df_disney_cleaned = df_disney.dropna(axis=1, how='all')

In [13]:
# Display the structure of the data franes after drop of empty columns
data = {
    'DataFrame': ['df_canyons_cleaned', 'df_miami_cleaned', 'df_vegas_cleaned', 'df_disney_cleaned'],
    'Rows': [df_canyons_cleaned.shape[0], df_miami_cleaned.shape[0], df_vegas_cleaned.shape[0], df_disney_cleaned.shape[0]],
    'Columns': [df_canyons_cleaned.shape[1], df_miami_cleaned.shape[1], df_vegas_cleaned.shape[1], df_disney_cleaned.shape[1]]
}

df_summary = pd.DataFrame(data)

print(df_summary)

            DataFrame    Rows  Columns
0  df_canyons_cleaned  102255       15
1    df_miami_cleaned   27490       14
2    df_vegas_cleaned  298682       20
3   df_disney_cleaned  247374       17


# Feature Selection

In [14]:
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

## Las Vegas data 

### Missing value imputation

In [15]:
column_names =df_vegas_cleaned.columns.tolist()

# Print the column names
print("Column Names:")
print(column_names)

Column Names:
['Las Vegas_STATION_NUM', 'Las Vegas_STATION_ID', 'Las Vegas_DATETIME', 'Las Vegas_air_temp_set_1', 'Las Vegas_pressure_set_1', 'Las Vegas_wind_speed_set_1', 'Las Vegas_solar_radiation_set_1', 'Las Vegas_precip_accum_set_1', 'Las Vegas_dew_point_temperature_set_1', 'Las Vegas_relative_humidity_set_1', 'Las Vegas_wind_direction_set_1', 'Las Vegas_wind_gust_set_1', 'Las Vegas_altimeter_set_1', 'Las Vegas_soil_temp_set_1', 'Las Vegas_visibility_set_1', 'Las Vegas_ceiling_set_1', 'YEAR', 'MONTH', 'DAY', 'HOUR']


In [16]:
print(df_vegas_cleaned.isnull().sum())

Las Vegas_STATION_NUM                         0
Las Vegas_STATION_ID                          0
Las Vegas_DATETIME                            0
Las Vegas_air_temp_set_1                      0
Las Vegas_pressure_set_1                 298659
Las Vegas_wind_speed_set_1                 1844
Las Vegas_solar_radiation_set_1          298659
Las Vegas_precip_accum_set_1               2310
Las Vegas_dew_point_temperature_set_1    297642
Las Vegas_relative_humidity_set_1             0
Las Vegas_wind_direction_set_1             1868
Las Vegas_wind_gust_set_1                286562
Las Vegas_altimeter_set_1                   144
Las Vegas_soil_temp_set_1                298659
Las Vegas_visibility_set_1                   23
Las Vegas_ceiling_set_1                  294237
YEAR                                          0
MONTH                                         0
DAY                                           0
HOUR                                          0
dtype: int64


In [17]:
# List of columns with missing values
columns_with_missing_values = [
    'Las Vegas_pressure_set_1', 'Las Vegas_wind_speed_set_1',
    'Las Vegas_solar_radiation_set_1', 'Las Vegas_precip_accum_set_1',
    'Las Vegas_dew_point_temperature_set_1', 'Las Vegas_wind_direction_set_1',
    'Las Vegas_wind_gust_set_1', 'Las Vegas_altimeter_set_1', 'Las Vegas_soil_temp_set_1', 'Las Vegas_visibility_set_1', 'Las Vegas_ceiling_set_1'
]

# Iterate over each column and impute missing values with the mean of the corresponding day and hour per available years
for column in columns_with_missing_values:
    if column in ['Las Vegas_pressure_set_1', 'Las Vegas_solar_radiation_set_1','Las Vegas_dew_point_temperature_set_1','Las Vegas_wind_gust_set_1', 'Las Vegas_soil_temp_set_1', 'Las Vegas_ceiling_set_1']:
        # Calculate the mean for each day and hour combination, excluding NaN values
        mean_per_day_hour = df_vegas_cleaned.groupby(['MONTH', 'DAY'])[column].transform('mean')
        # Use loc to set values in the original DataFrame
        df_vegas_cleaned.loc[:, column] = df_vegas_cleaned[column].fillna(mean_per_day_hour)
    else:
        # For other columns, impute missing values using the mean of the corresponding year month day and hour
        df_vegas_cleaned.loc[:, column] = df_vegas_cleaned.groupby(['MONTH', 'DAY', 'HOUR'])[column].transform(lambda x: x.fillna(x.mean()))

In [18]:
print(df_vegas_cleaned.isnull().sum())

Las Vegas_STATION_NUM                         0
Las Vegas_STATION_ID                          0
Las Vegas_DATETIME                            0
Las Vegas_air_temp_set_1                      0
Las Vegas_pressure_set_1                 297341
Las Vegas_wind_speed_set_1                    0
Las Vegas_solar_radiation_set_1          297341
Las Vegas_precip_accum_set_1                  0
Las Vegas_dew_point_temperature_set_1    266761
Las Vegas_relative_humidity_set_1             0
Las Vegas_wind_direction_set_1                0
Las Vegas_wind_gust_set_1                 58696
Las Vegas_altimeter_set_1                     0
Las Vegas_soil_temp_set_1                297341
Las Vegas_visibility_set_1                    0
Las Vegas_ceiling_set_1                  236180
YEAR                                          0
MONTH                                         0
DAY                                           0
HOUR                                          0
dtype: int64


In [19]:
# Forward fill missing values for Las Vegas dataset
columns_to_fill_forward = ['Las Vegas_pressure_set_1', 'Las Vegas_solar_radiation_set_1', 
                            'Las Vegas_dew_point_temperature_set_1', 'Las Vegas_soil_temp_set_1',
                            'Las Vegas_ceiling_set_1', 'Las Vegas_wind_gust_set_1']

for column in columns_to_fill_forward:
    df_vegas_cleaned.loc[:, column] = df_vegas_cleaned[column].fillna(method='ffill')

# Backward fill missing values for Las Vegas dataset
columns_to_fill_backward = ['Las Vegas_pressure_set_1', 'Las Vegas_solar_radiation_set_1', 
                             'Las Vegas_dew_point_temperature_set_1', 'Las Vegas_soil_temp_set_1',
                             'Las Vegas_ceiling_set_1', 'Las Vegas_wind_gust_set_1']

for column in columns_to_fill_backward:
    df_vegas_cleaned.loc[:, column] = df_vegas_cleaned[column].fillna(method='bfill')

# Check for remaining missing values
print(df_vegas_cleaned[columns_with_missing_values].isnull().sum())

Las Vegas_pressure_set_1                 0
Las Vegas_wind_speed_set_1               0
Las Vegas_solar_radiation_set_1          0
Las Vegas_precip_accum_set_1             0
Las Vegas_dew_point_temperature_set_1    0
Las Vegas_wind_direction_set_1           0
Las Vegas_wind_gust_set_1                0
Las Vegas_altimeter_set_1                0
Las Vegas_soil_temp_set_1                0
Las Vegas_visibility_set_1               0
Las Vegas_ceiling_set_1                  0
dtype: int64


### Feature Selection Using Linear Regression approach

In [20]:
# Drop the 'Canyons Ski Resort_DATETIME' column
df_vegas_cleaned = df_vegas_cleaned.drop('Las Vegas_DATETIME', axis=1)

In [21]:
# List of target variables
target_variables = ['Las Vegas_air_temp_set_1', 'Las Vegas_wind_speed_set_1', 
                    'Las Vegas_solar_radiation_set_1', 'Las Vegas_precip_accum_set_1']

# Iterate over each target variable
for target_variable in target_variables:
    # Use the other target variables (excluding the current target) as features
    features = df_vegas_cleaned.drop(['Las Vegas_STATION_NUM', 'Las Vegas_STATION_ID', 'YEAR', 'MONTH', 'DAY', 'HOUR'] + [target_variable], axis=1)

    # Extract the current target variable
    y = df_vegas_cleaned[target_variable]

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.2, random_state=42)

    # Train a linear regression model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Evaluate the model
    mse = mean_squared_error(y_test, y_pred)
    print(f'Mean Squared Error for predicting {target_variable}: {mse}')

    # Display coefficients or feature importances
    coefficients = pd.DataFrame({'Feature': features.columns, 'Coefficient': model.coef_})
    coefficients = coefficients.sort_values(by='Coefficient', ascending=False)

    print(f"\nFeature importances for predicting '{target_variable}':")
    print(coefficients)
    print('\n' + '-'*50 + '\n')

Mean Squared Error for predicting Las Vegas_air_temp_set_1: 12.98500821127231

Feature importances for predicting 'Las Vegas_air_temp_set_1':
                                  Feature  Coefficient
10             Las Vegas_visibility_set_1     0.579355
1              Las Vegas_wind_speed_set_1     0.149983
7               Las Vegas_wind_gust_set_1     0.099487
9               Las Vegas_soil_temp_set_1     0.035561
3            Las Vegas_precip_accum_set_1     0.008770
11                Las Vegas_ceiling_set_1     0.000229
0                Las Vegas_pressure_set_1     0.000021
8               Las Vegas_altimeter_set_1    -0.000022
6          Las Vegas_wind_direction_set_1    -0.000360
2         Las Vegas_solar_radiation_set_1    -0.003386
5       Las Vegas_relative_humidity_set_1    -0.178072
4   Las Vegas_dew_point_temperature_set_1    -1.588918

--------------------------------------------------

Mean Squared Error for predicting Las Vegas_wind_speed_set_1: 0.6712514479533299

Feature 

## Miami Data 

### Missing value imputation

In [22]:
column_names =df_miami_cleaned.columns.tolist()

# Print the column names
print("Column Names:")
print(column_names)

Column Names:
['Miami Beach_STATION_NUM', 'Miami Beach_STATION_ID', 'Miami Beach_DATETIME', 'Miami Beach_air_temp_set_1', 'Miami Beach_wind_speed_set_1', 'Miami Beach_solar_radiation_set_1', 'Miami Beach_relative_humidity_set_1', 'Miami Beach_wind_direction_set_1', 'Miami Beach_wind_gust_set_1', 'Miami Beach_altimeter_set_1', 'YEAR', 'MONTH', 'DAY', 'HOUR']


In [23]:
print(df_miami_cleaned.isnull().sum())

Miami Beach_STATION_NUM                    0
Miami Beach_STATION_ID                     0
Miami Beach_DATETIME                       0
Miami Beach_air_temp_set_1                 0
Miami Beach_wind_speed_set_1             210
Miami Beach_solar_radiation_set_1      26351
Miami Beach_relative_humidity_set_1      210
Miami Beach_wind_direction_set_1       13631
Miami Beach_wind_gust_set_1              210
Miami Beach_altimeter_set_1              210
YEAR                                       0
MONTH                                      0
DAY                                        0
HOUR                                       0
dtype: int64


In [24]:
#Checking for which dates we do have data
valid_data_combinations = df_miami_cleaned.dropna(subset=['Miami Beach_solar_radiation_set_1'])[['YEAR','MONTH', 'DAY', 'HOUR']].drop_duplicates()
print(valid_data_combinations)

       YEAR  MONTH  DAY  HOUR
13603  2022      6    5    14
13604  2022      6    5    15
13605  2022      6    5    16
22837  2023      6    2     8
22838  2023      6    2     9
22839  2023      6    2    10
22840  2023      6    2    11
22841  2023      6    2    12
22842  2023      6    2    13
22843  2023      6    2    14
22844  2023      6    2    15
22845  2023      6    2    16
22846  2023      6    2    17
22873  2023      6    3    14
22874  2023      6    3    15
22875  2023      6    3    16
22876  2023      6    3    17
23167  2023      6    8     8
23168  2023      6    8     9
23169  2023      6    8    10
23170  2023      6    8    11
23171  2023      6    8    12
23172  2023      6    8    13
23173  2023      6    8    14
23174  2023      6    8    15
23175  2023      6    8    16
23176  2023      6    8    17
23257  2023      6    9     8
23258  2023      6    9     9
23259  2023      6    9    10
23260  2023      6    9    11
23261  2023      6    9    12
23262  202

In [25]:
# List of columns with missing values
columns_with_missing_values = [
    'Miami Beach_air_temp_set_1', 'Miami Beach_wind_speed_set_1', 'Miami Beach_solar_radiation_set_1', 'Miami Beach_relative_humidity_set_1', 'Miami Beach_wind_direction_set_1', 'Miami Beach_wind_gust_set_1', 'Miami Beach_altimeter_set_1'
]

# Iterate over each column and impute missing values with the mean of the corresponding day and hour per available years
for column in columns_with_missing_values:
    if column in ['Miami Beach_solar_radiation_set_1', 'Miami Beach_wind_direction_set_1']:
        # Calculate the mean for each day and hour combination, excluding NaN values
        mean_per_day_hour = df_miami_cleaned.groupby(['MONTH', 'DAY'])[column].transform('mean')
        # Use loc to set values in the original DataFrame
        df_miami_cleaned.loc[:, column] = df_miami_cleaned[column].fillna(mean_per_day_hour)
    else:
        # For other columns, impute missing values using the mean of the corresponding day and hour
        df_miami_cleaned.loc[:, column] = df_miami_cleaned.groupby(['MONTH', 'DAY'])[column].transform(lambda x: x.fillna(x.mean()))

In [26]:
# Impute missing values for 'Miami Beach_solar_radiation_set_1' using backward fill
df_miami_cleaned['Miami Beach_solar_radiation_set_1'] = df_miami_cleaned['Miami Beach_solar_radiation_set_1'].fillna(method='bfill')
# Forward fill missing values
df_miami_cleaned['Miami Beach_solar_radiation_set_1'] = df_miami_cleaned['Miami Beach_solar_radiation_set_1'].fillna(method='ffill')

# Check for remaining missing values
print(df_miami_cleaned[columns_with_missing_values].isnull().sum())

Miami Beach_air_temp_set_1             0
Miami Beach_wind_speed_set_1           0
Miami Beach_solar_radiation_set_1      0
Miami Beach_relative_humidity_set_1    0
Miami Beach_wind_direction_set_1       0
Miami Beach_wind_gust_set_1            0
Miami Beach_altimeter_set_1            0
dtype: int64


<ipython-input-26-103d4fe3bdc2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_miami_cleaned['Miami Beach_solar_radiation_set_1'] = df_miami_cleaned['Miami Beach_solar_radiation_set_1'].fillna(method='bfill')
<ipython-input-26-103d4fe3bdc2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_miami_cleaned['Miami Beach_solar_radiation_set_1'] = df_miami_cleaned['Miami Beach_solar_radiation_set_1'].fillna(method='ffill')


### Feature Selection Using Linear Regression approach

In [27]:
# Drop the 'Canyons Ski Resort_DATETIME' column
df_miami_cleaned = df_miami_cleaned.drop('Miami Beach_DATETIME', axis=1)

In [28]:
# List of target variables
target_variables = ['Miami Beach_air_temp_set_1', 'Miami Beach_wind_speed_set_1', 
                    'Miami Beach_solar_radiation_set_1']

# Independent variables
independent_variables = df_miami_cleaned.drop(['Miami Beach_STATION_NUM', 'Miami Beach_STATION_ID', 'YEAR', 'MONTH', 'DAY', 'HOUR'], axis=1)

# Iterate over each target variable
for target_variable in target_variables:
    # Extract the current target variable
    y = df_miami_cleaned[target_variable]

    # Use the other target variables as features
    X = df_miami_cleaned.drop([target_variable] + ['Miami Beach_STATION_NUM', 'Miami Beach_STATION_ID', 'YEAR', 'MONTH', 'DAY', 'HOUR'], axis=1)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train a linear regression model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Evaluate the model
    mse = mean_squared_error(y_test, y_pred)
    print(f'Mean Squared Error for predicting {target_variable}: {mse}')

    # Display coefficients or feature importances
    coefficients = pd.DataFrame({'Feature': X.columns, 'Coefficient': model.coef_})
    coefficients = coefficients.sort_values(by='Coefficient', ascending=False)

    print(f"\nFeature importances for predicting '{target_variable}':")
    print(coefficients)
    print('\n' + '-'*50 + '\n')  # Just to separate the results for different target variables

Mean Squared Error for predicting Miami Beach_air_temp_set_1: 2.386886580300502

Feature importances for predicting 'Miami Beach_air_temp_set_1':
                               Feature  Coefficient
4          Miami Beach_wind_gust_set_1     0.185677
3     Miami Beach_wind_direction_set_1     0.004779
1    Miami Beach_solar_radiation_set_1     0.001458
5          Miami Beach_altimeter_set_1     0.000214
0         Miami Beach_wind_speed_set_1    -0.138576
2  Miami Beach_relative_humidity_set_1    -0.249995

--------------------------------------------------

Mean Squared Error for predicting Miami Beach_wind_speed_set_1: 0.02063531350155891

Feature importances for predicting 'Miami Beach_wind_speed_set_1':
                               Feature  Coefficient
4          Miami Beach_wind_gust_set_1     0.259381
3     Miami Beach_wind_direction_set_1     0.001486
1    Miami Beach_solar_radiation_set_1     0.000065
5          Miami Beach_altimeter_set_1     0.000009
0           Miami Beach_a

## Canyons data 

### Missing value imputation

In [29]:
column_names =df_canyons_cleaned.columns.tolist()

# Print the column names
print("Column Names:")
print(column_names)

Column Names:
['Canyons Ski Resort_STATION_NUM', 'Canyons Ski Resort_STATION_ID', 'Canyons Ski Resort_DATETIME', 'Canyons Ski Resort_air_temp_set_1', 'Canyons Ski Resort_wind_speed_set_1', 'Canyons Ski Resort_solar_radiation_set_1', 'Canyons Ski Resort_precip_accum_set_1', 'Canyons Ski Resort_relative_humidity_set_1', 'Canyons Ski Resort_wind_direction_set_1', 'Canyons Ski Resort_wind_gust_set_1', 'Canyons Ski Resort_soil_temp_set_1', 'YEAR', 'MONTH', 'DAY', 'HOUR']


In [30]:
print(df_canyons_cleaned.isnull().sum())

Canyons Ski Resort_STATION_NUM                    0
Canyons Ski Resort_STATION_ID                     0
Canyons Ski Resort_DATETIME                       0
Canyons Ski Resort_air_temp_set_1                 0
Canyons Ski Resort_wind_speed_set_1               0
Canyons Ski Resort_solar_radiation_set_1      83467
Canyons Ski Resort_precip_accum_set_1         71554
Canyons Ski Resort_relative_humidity_set_1        0
Canyons Ski Resort_wind_direction_set_1        1239
Canyons Ski Resort_wind_gust_set_1                0
Canyons Ski Resort_soil_temp_set_1            66103
YEAR                                              0
MONTH                                             0
DAY                                               0
HOUR                                              0
dtype: int64


In [31]:
# List of columns with missing values
columns_with_missing_values = [
    'Canyons Ski Resort_air_temp_set_1', 'Canyons Ski Resort_wind_speed_set_1',
    'Canyons Ski Resort_solar_radiation_set_1', 'Canyons Ski Resort_precip_accum_set_1',
    'Canyons Ski Resort_relative_humidity_set_1', 'Canyons Ski Resort_wind_direction_set_1',
    'Canyons Ski Resort_wind_gust_set_1', 'Canyons Ski Resort_soil_temp_set_1'
]

# Iterate over each column and impute missing values with the mean of the corresponding day and hour per available years
for column in columns_with_missing_values:
    if column in ['Canyons Ski Resort_solar_radiation_set_1', 'Canyons Ski Resort_soil_temp_set_1','Canyons Ski Resort_precip_accum_set_1','Canyons Ski Resort_wind_direction_set_1']:
        # Calculate the mean for each day and hour combination, excluding NaN values
        mean_per_day_hour = df_canyons_cleaned.groupby(['MONTH', 'DAY', 'HOUR'])[column].transform('mean')
        # Use loc to set values in the original DataFrame
        df_canyons_cleaned.loc[:, column] = df_canyons_cleaned[column].fillna(mean_per_day_hour)
    else:
        # For other columns, impute missing values using the mean of the corresponding day and hour
        df_canyons_cleaned.loc[:, column] = df_canyons_cleaned.groupby(['YEAR','MONTH', 'DAY', 'HOUR'])[column].transform(lambda x: x.fillna(x.mean()))

In [32]:
df_canyons_cleaned.head(500)

,Canyons Ski Resort_STATION_NUM,Canyons Ski Resort_STATION_ID,Canyons Ski Resort_DATETIME,Canyons Ski Resort_air_temp_set_1,Canyons Ski Resort_wind_speed_set_1,Canyons Ski Resort_solar_radiation_set_1,Canyons Ski Resort_precip_accum_set_1,Canyons Ski Resort_relative_humidity_set_1,Canyons Ski Resort_wind_direction_set_1,Canyons Ski Resort_wind_gust_set_1,Canyons Ski Resort_soil_temp_set_1,YEAR,MONTH,DAY,HOUR
0,4,SNC,2019-06-01 08:00:00,8.982500,0.467500,0.000000,263.652,73.205000,266.500000,1.600000,17.132184,2019,6,1,8
1,4,SNC,2019-06-01 09:00:00,7.517500,0.515000,0.000000,263.652,78.300000,232.250000,1.097500,17.061582,2019,6,1,9
2,4,SNC,2019-06-01 10:00:00,6.357500,0.337500,0.001694,263.652,80.602500,184.500000,0.875000,16.976551,2019,6,1,10
3,4,SNC,2019-06-01 11:00:00,5.620000,0.292500,0.133014,263.652,82.182500,244.750000,0.895000,16.877703,2019,6,1,11
4,4,SNC,2019-06-01 12:00:00,6.065000,0.560000,15.434694,263.652,80.507500,214.750000,0.840000,16.768013,2019,6,1,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2498,CCD,2019-06-01 08:00:00,7.626923,2.528462,0.000000,263.652,68.513077,246.326923,5.475000,17.132184,2019,6,1,8
496,2498,CCD,2019-06-01 09:00:00,6.683077,1.115769,0.000000,263.652,72.815385,157.774615,2.533077,17.061582,2019,6,1,9
497,2498,CCD,2019-06-01 10:00:00,6.489615,1.173462,0.001694,263.652,70.368077,128.635385,2.011923,16.976551,2019,6,1,10
498,2498,CCD,2019-06-01 11:00:00,5.758462,1.012308,0.133014,263.652,72.040385,177.577308,1.783077,16.877703,2019,6,1,11


### Feature Selection Using Linear Regression approach

In [33]:
print(df_canyons_cleaned.isnull().sum())

Canyons Ski Resort_STATION_NUM                0
Canyons Ski Resort_STATION_ID                 0
Canyons Ski Resort_DATETIME                   0
Canyons Ski Resort_air_temp_set_1             0
Canyons Ski Resort_wind_speed_set_1           0
Canyons Ski Resort_solar_radiation_set_1      0
Canyons Ski Resort_precip_accum_set_1         0
Canyons Ski Resort_relative_humidity_set_1    0
Canyons Ski Resort_wind_direction_set_1       0
Canyons Ski Resort_wind_gust_set_1            0
Canyons Ski Resort_soil_temp_set_1            0
YEAR                                          0
MONTH                                         0
DAY                                           0
HOUR                                          0
dtype: int64


In [34]:
# Drop the 'Canyons Ski Resort_DATETIME' column
df_canyons_cleaned = df_canyons_cleaned.drop('Canyons Ski Resort_DATETIME', axis=1)

In [35]:
# List of target variables
target_variables = ['Canyons Ski Resort_air_temp_set_1', 'Canyons Ski Resort_wind_speed_set_1', 
                    'Canyons Ski Resort_solar_radiation_set_1', 'Canyons Ski Resort_precip_accum_set_1']

# Iterate over each target variable
for target_variable in target_variables:
    # Use the other target variables (excluding the current target) as features
    features = df_canyons_cleaned.drop(['Canyons Ski Resort_STATION_NUM', 'Canyons Ski Resort_STATION_ID', 'YEAR', 'MONTH', 'DAY', 'HOUR'] + [target_variable], axis=1)

    # Extract the current target variable
    y = df_canyons_cleaned[target_variable]

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.2, random_state=42)

    # Train a linear regression model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Evaluate the model
    mse = mean_squared_error(y_test, y_pred)
    print(f'Mean Squared Error for predicting {target_variable}: {mse}')

    # Display coefficients or feature importances
    coefficients = pd.DataFrame({'Feature': features.columns, 'Coefficient': model.coef_})
    coefficients = coefficients.sort_values(by='Coefficient', ascending=False)

    print(f"\nFeature importances for predicting '{target_variable}':")
    print(coefficients)
    print('\n' + '-'*50 + '\n')

Mean Squared Error for predicting Canyons Ski Resort_air_temp_set_1: 10.478039817286936

Feature importances for predicting 'Canyons Ski Resort_air_temp_set_1':
                                      Feature  Coefficient
0         Canyons Ski Resort_wind_speed_set_1     0.584349
6          Canyons Ski Resort_soil_temp_set_1     0.282801
1    Canyons Ski Resort_solar_radiation_set_1     0.009458
4     Canyons Ski Resort_wind_direction_set_1    -0.002881
5          Canyons Ski Resort_wind_gust_set_1    -0.136269
2       Canyons Ski Resort_precip_accum_set_1    -0.151800
3  Canyons Ski Resort_relative_humidity_set_1    -0.176426

--------------------------------------------------

Mean Squared Error for predicting Canyons Ski Resort_wind_speed_set_1: 0.09230494719161915

Feature importances for predicting 'Canyons Ski Resort_wind_speed_set_1':
                                      Feature  Coefficient
5          Canyons Ski Resort_wind_gust_set_1     0.524831
0           Canyons Ski Resort

## Disney Data 

### Missing value imputation

In [36]:
column_names =df_disney_cleaned.columns.tolist()

# Print the column names
print("Column Names:")
print(column_names)

Column Names:
['Walt Disney Parks & Resorts_STATION_NUM', 'Walt Disney Parks & Resorts_STATION_ID', 'Walt Disney Parks & Resorts_DATETIME', 'Walt Disney Parks & Resorts_air_temp_set_1', 'Walt Disney Parks & Resorts_wind_speed_set_1', 'Walt Disney Parks & Resorts_precip_accum_set_1', 'Walt Disney Parks & Resorts_dew_point_temperature_set_1', 'Walt Disney Parks & Resorts_relative_humidity_set_1', 'Walt Disney Parks & Resorts_wind_direction_set_1', 'Walt Disney Parks & Resorts_wind_gust_set_1', 'Walt Disney Parks & Resorts_altimeter_set_1', 'Walt Disney Parks & Resorts_visibility_set_1', 'Walt Disney Parks & Resorts_ceiling_set_1', 'YEAR', 'MONTH', 'DAY', 'HOUR']


In [37]:
print(df_disney_cleaned.isnull().sum())

Walt Disney Parks & Resorts_STATION_NUM                         0
Walt Disney Parks & Resorts_STATION_ID                          0
Walt Disney Parks & Resorts_DATETIME                            0
Walt Disney Parks & Resorts_air_temp_set_1                     23
Walt Disney Parks & Resorts_wind_speed_set_1                  886
Walt Disney Parks & Resorts_precip_accum_set_1              76225
Walt Disney Parks & Resorts_dew_point_temperature_set_1    247033
Walt Disney Parks & Resorts_relative_humidity_set_1            87
Walt Disney Parks & Resorts_wind_direction_set_1              886
Walt Disney Parks & Resorts_wind_gust_set_1                246037
Walt Disney Parks & Resorts_altimeter_set_1                 12656
Walt Disney Parks & Resorts_visibility_set_1                  153
Walt Disney Parks & Resorts_ceiling_set_1                  183842
YEAR                                                            0
MONTH                                                           0
DAY       

In [38]:
# List of columns with missing values
columns_with_missing_values = [
    'Walt Disney Parks & Resorts_air_temp_set_1', 'Walt Disney Parks & Resorts_wind_speed_set_1',
    'Walt Disney Parks & Resorts_precip_accum_set_1', 'Walt Disney Parks & Resorts_dew_point_temperature_set_1',
    'Walt Disney Parks & Resorts_relative_humidity_set_1', 'Walt Disney Parks & Resorts_wind_direction_set_1',
    'Walt Disney Parks & Resorts_wind_gust_set_1', 'Walt Disney Parks & Resorts_altimeter_set_1', 'Walt Disney Parks & Resorts_visibility_set_1', 'Walt Disney Parks & Resorts_ceiling_set_1'
]

# Iterate over each column and impute missing values with the mean of the corresponding day and hour per available years
for column in columns_with_missing_values:
    if column in ['Walt Disney Parks & Resorts_dew_point_temperature_set_1', 'Walt Disney Parks & Resorts_wind_gust_set_1','Walt Disney Parks & Resorts_precip_accum_set_1', 'Walt Disney Parks & Resorts_ceiling_set_1']:
        # Calculate the mean for each day and hour combination, excluding NaN values
        mean_per_day_hour = df_disney_cleaned.groupby(['MONTH', 'DAY'])[column].transform('mean')
        # Use loc to set values in the original DataFrame
        df_disney_cleaned.loc[:, column] = df_disney_cleaned[column].fillna(mean_per_day_hour)
    else:
        # For other columns, impute missing values using the mean of the corresponding day and hour
        df_disney_cleaned.loc[:, column] = df_disney_cleaned.groupby(['YEAR','MONTH', 'DAY', 'HOUR'])[column].transform(lambda x: x.fillna(x.mean()))

In [39]:
print(df_disney_cleaned.isnull().sum())

Walt Disney Parks & Resorts_STATION_NUM                         0
Walt Disney Parks & Resorts_STATION_ID                          0
Walt Disney Parks & Resorts_DATETIME                            0
Walt Disney Parks & Resorts_air_temp_set_1                     23
Walt Disney Parks & Resorts_wind_speed_set_1                  886
Walt Disney Parks & Resorts_precip_accum_set_1                  0
Walt Disney Parks & Resorts_dew_point_temperature_set_1    245705
Walt Disney Parks & Resorts_relative_humidity_set_1            87
Walt Disney Parks & Resorts_wind_direction_set_1              886
Walt Disney Parks & Resorts_wind_gust_set_1                208480
Walt Disney Parks & Resorts_altimeter_set_1                 12656
Walt Disney Parks & Resorts_visibility_set_1                  153
Walt Disney Parks & Resorts_ceiling_set_1                       0
YEAR                                                            0
MONTH                                                           0
DAY       

In [40]:
# Forward fill missing values for Las Vegas dataset
columns_to_fill_forward = ['Walt Disney Parks & Resorts_air_temp_set_1', 'Walt Disney Parks & Resorts_wind_speed_set_1', 
                            'Walt Disney Parks & Resorts_dew_point_temperature_set_1', 'Walt Disney Parks & Resorts_relative_humidity_set_1',
                            'Walt Disney Parks & Resorts_wind_direction_set_1', 'Walt Disney Parks & Resorts_wind_gust_set_1','Walt Disney Parks & Resorts_altimeter_set_1','Walt Disney Parks & Resorts_visibility_set_1']

for column in columns_to_fill_forward:
    df_disney_cleaned.loc[:, column] = df_disney_cleaned[column].fillna(method='ffill')

# Backward fill missing values for Las Vegas dataset
columns_to_fill_backward = ['Walt Disney Parks & Resorts_dew_point_temperature_set_1', 'Walt Disney Parks & Resorts_wind_gust_set_1', 
                            ]

for column in columns_to_fill_backward:
    df_disney_cleaned.loc[:, column] = df_disney_cleaned[column].fillna(method='bfill')

# Check for remaining missing values
print(df_disney_cleaned[columns_with_missing_values].isnull().sum())

Walt Disney Parks & Resorts_air_temp_set_1                 0
Walt Disney Parks & Resorts_wind_speed_set_1               0
Walt Disney Parks & Resorts_precip_accum_set_1             0
Walt Disney Parks & Resorts_dew_point_temperature_set_1    0
Walt Disney Parks & Resorts_relative_humidity_set_1        0
Walt Disney Parks & Resorts_wind_direction_set_1           0
Walt Disney Parks & Resorts_wind_gust_set_1                0
Walt Disney Parks & Resorts_altimeter_set_1                0
Walt Disney Parks & Resorts_visibility_set_1               0
Walt Disney Parks & Resorts_ceiling_set_1                  0
dtype: int64


### Feature Selection Using Linear Regression approach

In [41]:
# Drop the 'Canyons Ski Resort_DATETIME' column
df_disney_cleaned = df_disney_cleaned.drop('Walt Disney Parks & Resorts_DATETIME', axis=1)

In [42]:
# List of target variables
target_variables = ['Walt Disney Parks & Resorts_air_temp_set_1', 'Walt Disney Parks & Resorts_wind_speed_set_1', 
                    'Walt Disney Parks & Resorts_precip_accum_set_1']

# Iterate over each target variable
for target_variable in target_variables:
    # Use the other target variables (excluding the current target) as features
    features = df_disney_cleaned.drop(['Walt Disney Parks & Resorts_STATION_NUM', 'Walt Disney Parks & Resorts_STATION_ID', 'YEAR', 'MONTH', 'DAY', 'HOUR'] + [target_variable], axis=1)

    # Extract the current target variable
    y = df_disney_cleaned[target_variable]

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.2, random_state=42)

    # Train a linear regression model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Evaluate the model
    mse = mean_squared_error(y_test, y_pred)
    print(f'Mean Squared Error for predicting {target_variable}: {mse}')

    # Display coefficients or feature importances
    coefficients = pd.DataFrame({'Feature': features.columns, 'Coefficient': model.coef_})
    coefficients = coefficients.sort_values(by='Coefficient', ascending=False)

    print(f"\nFeature importances for predicting '{target_variable}':")
    print(coefficients)
    print('\n' + '-'*50 + '\n')

Mean Squared Error for predicting Walt Disney Parks & Resorts_air_temp_set_1: 3.339581543795612

Feature importances for predicting 'Walt Disney Parks & Resorts_air_temp_set_1':
                                             Feature  Coefficient
0       Walt Disney Parks & Resorts_wind_speed_set_1     0.150702
1     Walt Disney Parks & Resorts_precip_accum_set_1     0.021160
4   Walt Disney Parks & Resorts_wind_direction_set_1     0.018447
2  Walt Disney Parks & Resorts_dew_point_temperat...     0.002271
6        Walt Disney Parks & Resorts_altimeter_set_1     0.000319
8          Walt Disney Parks & Resorts_ceiling_set_1    -0.001326
7       Walt Disney Parks & Resorts_visibility_set_1    -0.021118
5        Walt Disney Parks & Resorts_wind_gust_set_1    -0.023924
3  Walt Disney Parks & Resorts_relative_humidity_...    -0.158474

--------------------------------------------------

Mean Squared Error for predicting Walt Disney Parks & Resorts_wind_speed_set_1: 0.4150171427729038

Feature i

# Save as parquet files

In [43]:
# Define the folder where you want to save the Parquet files
output_folder = 'C:\\Users\\clair\\MasterThesis\\Feature_selected_data\\'

# Make sure the output folder exists, create it if not
os.makedirs(output_folder, exist_ok=True)

# Specify the file names you want for each DataFrame
file_names = {
    'df_miami_cleaned': 'miami_data.parquet',
    'df_vegas_cleaned': 'vegas_data.parquet',
    'df_canyons_cleaned': 'canyons_data.parquet',
    'df_disney_cleaned': 'disney_data.parquet'
}

# Save each DataFrame to a separate Parquet file
for df_name, file_name in file_names.items():
    df = globals()[df_name]  # Assuming your DataFrames are in the global namespace
    df.to_parquet(os.path.join(output_folder, file_name))

print("Parquet files saved successfully.")

Parquet files saved successfully.
